# Air Pollution Prediction (Use Case) - SENEM AKTAŞ

# 1- Read Dataset CSV 

# 2- Preprocessing Dataset

# 3- Visualisation Dataset

# 4- Machine Learninig Model

# 5- Deep Learning Model

# 6- Evaluation

# 7- Result - Comments